In [ ]:
!pip install googledrivedownloader

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install openpyxl

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import RandomizedSearchCV, cross_validate
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor

import seaborn as sns
import scipy.stats as st

from google_drive_downloader import GoogleDriveDownloader as gdd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#changable parameters
target = "Class Label(GDM /Non GDM)"
test_size = 0.2


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
#import dataset from google drive
"""gdd.download_file_from_google_drive(file_id=data_url.split('id=')[1][:21],
                                    dest_path='/content/drive/MyDrive/Projects/gestational diabetes/data.xlsx',
                                    unzip=True)"""

"gdd.download_file_from_google_drive(file_id=data_url.split('id=')[1][:21],\n                                    dest_path='/content/drive/MyDrive/Projects/gestational diabetes/data.xlsx',\n                                    unzip=True)"

In [4]:
df = pd.read_excel("data.xlsx",engine='openpyxl')
df.fillna(method="ffill", inplace=True,axis=0)
df.drop(['Case Number','BMI', 'OGTT'],inplace=True, axis=1)
df.reset_index(inplace=True)
df.head()

,index,Age,No of Pregnancy,Gestation in previous Pregnancy,HDL,Family History,unexplained prenetal loss,Large Child or Birth Default,PCOS,Sys BP,Dia BP,Hemoglobin,Sedentary Lifestyle,Prediabetes,Class Label(GDM /Non GDM)
0,0,22,2,1,55.0,0,0,0,0,102.0,69,12.0,0,0,0
1,1,26,2,1,53.0,0,0,0,0,101.0,63,12.4,0,0,0
2,2,29,1,0,50.0,0,0,0,0,118.0,79,14.3,0,0,0
3,3,28,2,1,51.0,0,0,0,0,99.0,70,15.0,0,0,0
4,4,21,2,1,52.0,0,0,0,0,116.0,65,15.0,0,0,0


In [5]:
len(df)

3525

In [6]:
#classification or regression
if (type(df[target][0]) == str) or (type(df[target][0]) == int) or (type(df[target][0]) == np.int16) or (type(df[target][0]) == np.int32) or (type(df[target][0]) == np.int64):
    models_type = 'classification'
else:
    models_type = 'regression'


print(models_type)

classification


In [7]:
def label_encoding(old_column):
    le = LabelEncoder()
    le.fit(old_column)
    new_column = le.transform(old_column)
    return new_column

In [8]:
#encoding string parameters
for i in df.columns:
    if type(df[i][0]) == str:
        df[i] = label_encoding(df[i])

In [9]:
#extracting x and y
y = df[target].values

x = df.drop([target], axis=1).values

In [10]:
#spliting  data
X_train, X_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=test_size)

In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
# Reshaping the input to be 3-dimensional (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [13]:
# Define LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1, activation='sigmoid'))  # Assuming binary classification, change activation function accordingly

In [14]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
89/89 [==============================] - 6s 16ms/step - loss: 0.4726 - accuracy: 0.9071 - val_loss: 0.1971 - val_accuracy: 0.9418
Epoch 2/50
89/89 [==============================] - 0s 5ms/step - loss: 0.1536 - accuracy: 0.9507 - val_loss: 0.0916 - val_accuracy: 0.9787
Epoch 3/50
89/89 [==============================] - 0s 5ms/step - loss: 0.1065 - accuracy: 0.9699 - val_loss: 0.0755 - val_accuracy: 0.9801
Epoch 4/50
89/89 [==============================] - 0s 5ms/step - loss: 0.0954 - accuracy: 0.9709 - val_loss: 0.0682 - val_accuracy: 0.9787
Epoch 5/50
89/89 [==============================] - 0s 5ms/step - loss: 0.0892 - accuracy: 0.9709 - val_loss: 0.0647 - val_accuracy: 0.9801
Epoch 6/50
89/89 [==============================] - 0s 5ms/step - loss: 0.0836 - accuracy: 0.9709 - val_loss: 0.0628 - val_accuracy: 0.9816
Epoch 7/50
89/89 [==============================] - 0s 6ms/step - loss: 0.0781 - accuracy: 0.9734 - val_loss: 0.0604 - val_accuracy: 0.9816
Epoch 8/50
89/89 [=

In [16]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

23/23 [==============================] - 0s 2ms/step - loss: 0.2189 - accuracy: 0.9716
Test Loss: 0.21894221007823944, Test Accuracy: 0.9716312289237976
